In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
from datasets import load_dataset
import pandas as pd

In [23]:
# Load train dataset
ds = load_dataset("ailsntua/QEvasion")

# Convert to pandas
df_train = ds["train"].to_pandas()

# Create dictionary with key as the annotator and value the Dataframe with
# only the corresponding sQAs
split_dfs = {
    category: group for category, group in df_train.groupby('annotator_id')
    }

In [19]:
split_dfs[85]

,title,date,president,url,interview_question,interview_answer,gpt3.5_summary,gpt3.5_prediction,question,label,annotator_id
0,"The President's News Conference in Hanoi, Vietnam","September 10, 2023",Joseph R. Biden,https://www.presidency.ucsb.edu/documents/the-...,\nQ. Of the Biden administration. And accused ...,"\nThe President. Well, look, first of all, the...",The question consists of 2 parts: \n1. How wou...,Question part: 1. How would you respond to the...,How would you respond to the accusation that t...,1.1 Explicit,85
1,"The President's News Conference in Hanoi, Vietnam","September 10, 2023",Joseph R. Biden,https://www.presidency.ucsb.edu/documents/the-...,\nQ. Of the Biden administration. And accused ...,"\nThe President. Well, look, first of all, the...",The question consists of 2 parts: \n1. How wou...,Question part: 1. How would you respond to the...,Do you think President Xi is being sincere abo...,2.4 General,85
2,"The President's News Conference in Hanoi, Vietnam","September 10, 2023",Joseph R. Biden,https://www.presidency.ucsb.edu/documents/the-...,\nQ. No worries. Do you believe the country's ...,"\nThe President. Look, I think China has a dif...",The question consists of two parts:\n\n1. Q1: ...,Question part: Q1 - Do you believe the country...,1. Q1: Do you believe the country's slowdown a...,2.3 Partial/half-answer,85
3,"The President's News Conference in Hanoi, Vietnam","September 10, 2023",Joseph R. Biden,https://www.presidency.ucsb.edu/documents/the-...,\nQ. No worries. Do you believe the country's ...,"\nThe President. Look, I think China has a dif...",The question consists of two parts:\n\n1. Q1: ...,Question part: Q1 - Do you believe the country...,2. Q2: Are you worried about the meeting betwe...,2.1 Dodging,85
4,"The President's News Conference in Hanoi, Vietnam","September 10, 2023",Joseph R. Biden,https://www.presidency.ucsb.edu/documents/the-...,"\nQ. I can imagine. It is evening, I'd like to...","\nThe President. Well, I hope I get to see Mr....",The question consists of 3 parts:\n1. Is the P...,Question part: 1. Is the President's engagemen...,Is the President's engagement with Asian coun...,1.1 Explicit,85
...,...,...,...,...,...,...,...,...,...,...,...
3064,The President's News Conference,"October 17, 2007",George W. Bush,https://www.presidency.ucsb.edu/documents/the-...,\nQ. Was Syria part of those talks? Is Syria p...,\nThe President. Proliferation is a part of th...,"The question consists of 2 parts:\n\n1. ""Was S...","Question part: 1. ""Was Syria part of those tal...",Is Syria part of the talks?,2.2 Deflection,85
3065,The President's News Conference,"October 17, 2007",George W. Bush,https://www.presidency.ucsb.edu/documents/the-...,\nQ. Including Syria?,"\nThe President. Elaine [Elaine Quijano, Cable...",The question consists of 1 part:\n\n1. Is Syri...,Question part: 1. Is Syria included in somethi...,Is Syria included in something?,2.6 Declining to answer,85
3170,The President's News Conference,"July 12, 2007",George W. Bush,https://www.presidency.ucsb.edu/documents/the-...,\nQ. Can I have a followup?,"\nThe President. Yes, you can, because you're ...",The question consists of 1 part:\n\n- Can I ha...,Question part: Can I have a followup?\nVerdict...,Can I have a followup?,1.1 Explicit,85
3192,The President's News Conference With Prime Min...,"June 09, 2007",George W. Bush,https://www.presidency.ucsb.edu/documents/the-...,\nQ. Deadline for the Kosovo independence?,\nPresident Bush. A decline?,There is only one part to this question: \n\n-...,Question part: 1. Deadline for the Kosovo inde...,Deadline for the Kosovo independence?,2.8 Clarification,85
